In [4]:
import torch as t
from torch import einsum
from einops import rearrange, reduce, repeat
import numpy as np
import mlab_tests
import days.day1_tests

In [5]:
import sys
print(sys.path)

['/home/ubuntu/mlab/max', '/home/ubuntu/.vscode-server/extensions/ms-toolsai.jupyter-2021.11.1001550889/pythonFiles/vscode_datascience_helpers', '/home/ubuntu/.vscode-server/extensions/ms-toolsai.jupyter-2021.11.1001550889/pythonFiles', '/home/ubuntu/.vscode-server/extensions/ms-toolsai.jupyter-2021.11.1001550889/pythonFiles/lib/python', '/usr/lib/python38.zip', '/usr/lib/python3.8', '/usr/lib/python3.8/lib-dynload', '', '/home/ubuntu/.local/lib/python3.8/site-packages', '/home/ubuntu/mlab', '/usr/local/lib/python3.8/dist-packages', '/usr/lib/python3/dist-packages', '/usr/lib/python3/dist-packages/IPython/extensions', '/home/ubuntu/.ipython']


In [6]:
# docs a little confusing

rearrange(torch.arange(3, 9), "(d1 d) -> d1 d", d1=3)

tensor([[3, 4],
        [5, 6],
        [7, 8]])

In [7]:
rearrange(torch.arange(1, 7), "(h w) -> h w", h = 2)

tensor([[1, 2, 3],
        [4, 5, 6]])

In [19]:
rearrange(torch.arange(1, 7), "(h w l) -> h w l", h = 1, l = 1)

tensor([[[1],
         [2],
         [3],
         [4],
         [5],
         [6]]])

In [18]:
t.tensor([[[1], [2], [3], [4], [5], [6]]]).shape

torch.Size([1, 6, 1])

In [35]:
def normalize(temps):
    temps = rearrange(temps, "(h w) -> h w", w = 7)
    print(temps)
    temps -= reduce(temps, "h w -> h 1", "mean") 
    return temps / reduce(temps, "h w -> h 1", t.std)
normalize(t.tensor([71,72,70,75,71,72,70,68,65,60,68,60,55,59,75,80,85,80,78,72,83], dtype=float))

tensor([[71., 72., 70., 75., 71., 72., 70.],
        [68., 65., 60., 68., 60., 55., 59.],
        [75., 80., 85., 80., 78., 72., 83.]], dtype=torch.float64)
tensor([[1.7182],
        [4.9473],
        [4.4721]], dtype=torch.float64)


tensor([[-0.3326,  0.2494, -0.9146,  1.9954, -0.3326,  0.2494, -0.9146],
        [ 1.1839,  0.5775, -0.4331,  1.1839, -0.4331, -1.4438, -0.6353],
        [-0.8944,  0.2236,  1.3416,  0.2236, -0.2236, -1.5652,  0.8944]],
       dtype=torch.float64)

In [36]:
def my_dot(t1, t2):
    return reduce(t1 * t2, "... d -> ...", "sum")
    

In [37]:
my_dot(t.Tensor([[1,1,0],[0,0,1]]),t.Tensor([[1,1,0],[1,1,0]]))

tensor([2., 0.])

In [45]:
def get_corners(h, w, n):
    top = t.linspace(0, h, n + 1)
    side = t.linspace(0, w, n + 1)
    rep_top = repeat(top, "w -> h w", h = n + 1)
    rep_side = repeat(side, "h -> h w", w = n + 1)
    return rearrange([rep_top, rep_side], "a b c -> (b c) a")
get_corners(9, 15, 3)

tensor([0., 3., 6., 9.]) tensor([ 0.,  5., 10., 15.])
tensor([[0., 3., 6., 9.],
        [0., 3., 6., 9.],
        [0., 3., 6., 9.],
        [0., 3., 6., 9.]]) tensor([[ 0.,  0.,  0.,  0.],
        [ 5.,  5.,  5.,  5.],
        [10., 10., 10., 10.],
        [15., 15., 15., 15.]])


tensor([[ 0.,  0.],
        [ 3.,  0.],
        [ 6.,  0.],
        [ 9.,  0.],
        [ 0.,  5.],
        [ 3.,  5.],
        [ 6.,  5.],
        [ 9.,  5.],
        [ 0., 10.],
        [ 3., 10.],
        [ 6., 10.],
        [ 9., 10.],
        [ 0., 15.],
        [ 3., 15.],
        [ 6., 15.],
        [ 9., 15.]])

In [203]:
def ex4(H: float, W: float, n: int):
    xaxis = torch.linspace(0, H, n + 1)
    xtile = torch.tile(xaxis, dims=(n + 1, 1))
    yaxis = torch.linspace(0, W, n + 1)[:, None]
    ytile = torch.tile(yaxis, dims=(n + 1,))
    return torch.stack([rearrange(xtile, "h w -> (h w)"), rearrange(ytile, "h w -> (h w)")]).T


ex4(15, 9, 3)

tensor([[ 0.,  5., 10., 15.],
        [ 0.,  5., 10., 15.],
        [ 0.,  5., 10., 15.],
        [ 0.,  5., 10., 15.]])


tensor([[ 0.,  0.],
        [ 5.,  0.],
        [10.,  0.],
        [15.,  0.],
        [ 0.,  3.],
        [ 5.,  3.],
        [10.,  3.],
        [15.,  3.],
        [ 0.,  6.],
        [ 5.,  6.],
        [10.,  6.],
        [15.,  6.],
        [ 0.,  9.],
        [ 5.,  9.],
        [10.,  9.],
        [15.,  9.]])

In [53]:
def eye(n):
    x = t.arange(n)
    first = x == rearrange(x, "d -> d 1")
    first = first.type(t.int)
    return (first)

eye(4)

tensor([[1, 0, 0, 0],
        [0, 1, 0, 0],
        [0, 0, 1, 0],
        [0, 0, 0, 1]], dtype=torch.int32)

In [69]:
def sample(p, n):
    cum = t.cumsum(p, dim = 0)
    print(cum)
    rand = t.rand((n, 1))
    print(rand)
    print(rand > cum)
    return(reduce(rand > cum, "h w -> h", "sum"))

# row vec and col vec give cartesian operations
    

In [70]:
sample(t.tensor([0.1, 0.3, 0.6]), 5)

tensor([0.1000, 0.4000, 1.0000])
tensor([[0.6129],
        [0.9464],
        [0.5580],
        [0.2154],
        [0.2052]])
tensor([[ True,  True, False],
        [ True,  True, False],
        [ True,  True, False],
        [ True, False, False],
        [ True, False, False]])


tensor([2, 2, 2, 1, 1])

In [71]:
def accuracy(scores, y):
    y_hat = reduce(scores, "h w -> h". t.argmax)
    return t.mean(y_hat == y)


In [72]:
def topk_acc(scores, y):
    equal = scores.argsort()[:, -k:] == y[:, None]
    return equal.any(dim = -1).mean()

In [75]:
def rev(p, i):
    return t.gather(p, dim = -1, index=i).sum()
rev(t.tensor([10, 20, 30, 40]), t.tensor([1, 1, 2]))

tensor(70)

In [76]:
def sample2(A, n):
    indices = t.randint(A.shape[1], (A.shape[0], n))
    return t.gather(A, dim = 1, index = indices)

values = t.rand((10, 5))
print(values)
print(sample2(values, 3))

tensor([[0.8982, 0.3731, 0.6732, 0.8753, 0.7616],
        [0.5981, 0.1035, 0.4055, 0.0884, 0.6497],
        [0.5420, 0.3132, 0.9956, 0.3145, 0.3447],
        [0.6885, 0.2266, 0.9304, 0.0089, 0.5238],
        [0.4605, 0.6628, 0.0478, 0.0785, 0.1228],
        [0.5382, 0.8775, 0.6688, 0.5643, 0.6476],
        [0.2065, 0.1864, 0.9762, 0.7217, 0.7781],
        [0.2293, 0.8629, 0.9419, 0.8488, 0.3082],
        [0.6801, 0.6376, 0.4772, 0.3736, 0.3509],
        [0.9329, 0.7375, 0.5130, 0.0025, 0.3274]])
tensor([[0.8753, 0.8753, 0.3731],
        [0.0884, 0.1035, 0.4055],
        [0.3447, 0.5420, 0.5420],
        [0.2266, 0.6885, 0.0089],
        [0.6628, 0.0478, 0.6628],
        [0.6688, 0.6688, 0.6688],
        [0.7781, 0.2065, 0.7781],
        [0.3082, 0.8629, 0.2293],
        [0.6801, 0.6801, 0.6801],
        [0.5130, 0.0025, 0.9329]])


In [102]:
def to_one_hot(v, K):
    out = t.zeros((v.shape[0], K)).to(int)
    print(out)
    indices = v[:, None].to(int)
    print(indices)
    values = t.ones_like(indices).to(int)
    print(values)
    return out.scatter(dim = -1, index = indices, src = values)

In [103]:
to_one_hot(t.tensor([3, 2, 4]), 5)

tensor([[0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0]])
tensor([[3],
        [2],
        [4]])
tensor([[1],
        [1],
        [1]])


tensor([[0, 0, 0, 1, 0],
        [0, 0, 1, 0, 0],
        [0, 0, 0, 0, 1]])

In [104]:
def relu(A):
    A = A.clone()
    A[A < 0] = 0
    return A

In [105]:
def dropout(A, p, is_train):
    if not is_train:
        return A
    A[t.rand_like(A) > p] = 0
    return A

In [106]:
a = torch.arange(6).reshape(2, 3)
b = torch.einsum('ij->ji', a)

tensor([[0, 3],
        [1, 4],
        [2, 5]])

In [107]:
def linear(input, weights, b):
    return t.einsum('ik, ...k -> ...i', [weights, input]) + b

In [109]:
def softmax(A):
    exps = t.exp(A)
    return exps / reduce(exps, "... d -> ... 1", "sum")

In [110]:
def logsm(A):
    B = A - t.max(A)
    return B - t.exp(B).sum(-1, keepdim=True).log()

In [ ]:
def centropy(A):
    probs = log

In [142]:
def convolve(v, w):
    outlen = len(v) - len(w) + 1
    reps = repeat(t.arange(len(w)), 'w -> h w', h = outlen)
    reps += t.arange(outlen)[:, None]
    return einsum("...ij, ...j -> ...i", [v[reps], w])


In [143]:
convolve(t.tensor([4, 5, 23, 23, 3, 2, -5, 6]), t.tensor([1, 2, 3]))

tensor([ 83, 120,  78,  35,  -8,  10])

In [ ]:
def convolve_fancy(v, w):
    outlen = v.shape[-1] - w.shape[-1] + 1
    reps = repeat(t.arange(len(w)), 'w -> h w', h = outlen)
    reps += t.arange(outlen)[:, None]
    return einsum("...ij, ...j -> ...i", [v[reps], w])

In [144]:
x = torch.randn(3, 3)
x

tensor([[-0.9211, -1.6007,  0.3251],
        [-0.2179, -0.4691,  0.6289],
        [-0.2132,  0.5033,  1.6385]])

In [150]:
x.as_strided((2, 5), (1, 1))

tensor([[-0.9211, -1.6007,  0.3251, -0.2179, -0.4691],
        [-1.6007,  0.3251, -0.2179, -0.4691,  0.6289]])

In [183]:
def convolve_fancy(v, w):
    batch_size, in_chans, seq_len = v.shape
    outlen = seq_len - w.shape[-1] + 1
    u = v.as_strided((batch_size, outlen, in_chans, w.shape[-1]), (in_chans * seq_len, 1, seq_len, 1))
    return einsum("...tik, oik -> ...ot", [u, w])

In [186]:
v = t.rand((2, 4))
print(v)
w = (t.rand(1, 2, 2) > 0.5).to(t.float)
print(w)
convolve_fancy(v[None, :], w)

tensor([[0.8000, 0.1375, 0.1684, 0.3943],
        [0.3807, 0.9407, 0.0206, 0.6891]])
tensor([[[1., 1.],
         [1., 1.]]])


tensor([[[2.2589, 1.2672, 1.2724]]])

In [187]:
def ex21(x, weight):
    x = rearrange(x, "b c s -> b s c")
    out_channels, _, kernel_size = weight.shape
    B, S, C = x.shape
    x = x.contiguous()
    strided_input = torch.as_strided(x, (B, S - kernel_size + 1, kernel_size, C, out_channels), (S * C, C, C, 1, 0), 0)
    weight_rearranged = rearrange(weight, "o i k -> k i o")
    added = strided_input * weight_rearranged
    summed = reduce(added, "b s c in_channels out_channels -> b s out_channels", "sum")
    summed = rearrange(summed, "b s c->b c s")
    return summed

ex21(v.unsqueeze(0), w)

tensor([[[2.2589, 1.2672, 1.2724]]])

In [198]:
def convolve_fanciest(v, w, padding = 0, stride = 1):
    batch_size, in_chans, seq_len = v.shape
    kernel_size = w.shape[-1]
    outlen = int((seq_len - kernel_size + padding * 2)/stride + 1)
    v = t.cat([t.zeros(batch_size, in_chans, padding), v, t.zeros(batch_size, in_chans, padding)], dim = -1)
    u = v.as_strided((batch_size, outlen, in_chans, kernel_size), (in_chans * seq_len, stride, seq_len, 1))
    return einsum("...tik, oik -> ...ot", [u, w])

In [201]:
convolve_fanciest(v.unsqueeze(0), w, padding = 10, stride =2)

tensor([[[0.0000, 0.0000, 0.0000, 0.9375, 0.5627, 0.9375, 0.5627, 0.0000,
          0.0000, 0.0000, 0.0000, 0.0000]]])

In [ ]:
convolve_fanciest()